In [19]:
import os
import os.path

In [60]:
raw_read_1 = "/media/eternus1/nfs/projects/evodevo/trim_testing/dataset/bacteria/ecoli_5/SRR519926_1.fastq"
read_1_trimmomatic = "/media/eternus1/nfs/projects/evodevo/trim_testing/dataset/bacteria/ecoli_5/trimmomatic_14/out_1P.fq"
reads_1_v2trim = "/media/eternus1/nfs/projects/evodevo/trim_testing/dataset/bacteria/ecoli_5/v2trim/trim.trim_1.fastq"

In [38]:
def reads_reader(reads_file_fastq):    
    reads = {}
    header = None
    with open(reads_file_fastq) as fh:
        for i, line in enumerate(fh):
            line = line.strip()
            if line.startswith("@SRR"):
                if header:
                    reads[header] = "".join(seq)
                header = line.split()[0][1:]
                seq = []
                header_switcher = True
            elif header_switcher:
                seq.append(line)
                header_switcher = False
    if header:
        reads[header] = "".join(seq)
    return reads

In [61]:
raw_reads = reads_reader(raw_read_1)
trimmomatic_reads = reads_reader(read_1_trimmomatic)
v2trim_reads = reads_reader(reads_1_v2trim)

In [62]:
different_reads = []
losted_reads = []
i = 0
for header, seq in trimmomatic_reads.items():
    if header in v2trim_reads.keys():
        if seq != v2trim_reads[header]:
            different_reads.append(header)
    else:
        losted_reads.append(header)

In [68]:
print(len(v2trim_reads[different_reads[0]]))
print(len(trimmomatic_reads[different_reads[0]]))

157
251


In [66]:
i = 0
for header, seq in v2trim_reads.items():
    if len(seq) == 251:
        i += 1

In [70]:
len(losted_reads)

4294

In [80]:
nontrimmed_by_trimmomatic = []
for key in different_reads:
    v2_seq = v2trim_reads[key]
    trim_seq = trimmomatic_reads[key]
    if len(v2_seq) > len(trim_seq):
        print(v2_seq.replace(trim_seq, ""))
    elif len(trim_seq) > len(v2_seq):
        nontrimmed_by_trimmomatic.append(trim_seq.replace(v2_seq, ""))

In [87]:
j = 0
for i in nontrimmed_by_trimmomatic:
    if "CTCGTATGCCGTCTTCTGCTTG" in i:
        j += 1
print(j)

21857
